SQLITE

In [1]:
import sqlite3
import os
try:
    import pyarrow
except:
    %pip install pyarrow
    import pyarrow

try:
    import pandas as pd
except:
    %pip install pandas
    import pandas as pd

import json
import re

print("done")

done


Loading files

In [2]:
class DButils:
    def __init__(self, path: str, filename: str, exists_ok: bool =False):
        """
        Initialize a new sqlite database.

        Args:
            path (str): Path to the sqlite database file.
            filename (str): name of the sqlite file.
            exists_ok (bool): will thow an error if the database already exists if exists_ok is False.

        Notes:
            If the file does not exist, it will be created.
        """
        
        if not exists_ok and \
            os.path.exists(os.path.join(path, filename)):
            raise FileExistsError(f"Database {filename} already exists at {path}.")

        os.makedirs(path, exist_ok=True)
        self.PATH_TO_DB = os.path.join(path, filename)

    def is_table(self, table_name: str) -> bool:
        with sqlite3.connect(self.PATH_TO_DB) as conn:
            cursor = conn.cursor()
            cursor.execute(
            "SELECT 1 FROM sqlite_master WHERE type='table' AND name=?", (table_name.upper(),)
            )
            return cursor.fetchone() is not None
        
    def create_table(self, table_name: str, schema: dict):
        schema_str = ", ".join(f"{col} {col_type}" for col, col_type in schema.items())

        with sqlite3.connect(self.PATH_TO_DB) as conn:
            cursor = conn.cursor()
            cursor.execute("PRAGMA foreign_keys = ON")
            cursor.execute(f"CREATE TABLE {table_name.upper()} ({schema_str})")
            return True
        
        
    def insert(self, table_name: str, data: dict[str, any]) -> bool:

        columns = ", ".join(data.keys())
        values = tuple(data.values())
        placeholders = ", ".join("?" for _ in data)
        
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        #print(sql)
        try:
            with sqlite3.connect(self.PATH_TO_DB) as conn:
                cursor = conn.cursor()
                cursor.execute("PRAGMA foreign_keys = ON")
                cursor.execute(sql, values)
                conn.commit()
                return True
        except sqlite3.Error as e:
            print(f"Error inserting data into table '{table_name}': {e}")
            return False
        
    def insert_restaurant(self, restaurant: pd.DataFrame):
        address = pd.DataFrame()
        address[["ADDRESS", "OTHER"]] = restaurant["address"].str.split(", ", expand=True)
        address[["POSTAL_CODE", "CITY", "COUNTRY"]] = address["OTHER"].str.split(" ", expand=True)
        address.drop(columns=["OTHER"], inplace=True)
        address = {key: value.iloc[0] for key, value in address.items()}

        restaurant_keys = [col for col in restaurant.columns if col not in ["address", "photos", "reviews", "geographic_location"]]
        restaurant_dict = {key: restaurant[key].iloc[0] for key in restaurant_keys}
        restaurant_dict["opening_hours"] = json.dumps(restaurant_dict["opening_hours"].tolist(), ensure_ascii=False)
        restaurant_dict["ranking"] = json.dumps(restaurant_dict["ranking"], ensure_ascii=False)
        restaurant_dict["detailed_rating"] = json.dumps(restaurant_dict["detailed_rating"], ensure_ascii=False)
        restaurant_dict["details"] = json.dumps({key: json.dumps(val.tolist(), ensure_ascii=False) for key, val in restaurant_dict["details"].items()}, ensure_ascii=False)
        restaurant_dict["address"] = address["ADDRESS"]
        restaurant_dict["postal_code"] = address["POSTAL_CODE"]
        restaurant_dict["city"] = address["CITY"]
        restaurant_dict["country"] = address["COUNTRY"]

        with sqlite3.connect(self.PATH_TO_DB) as conn:
            cursor = conn.cursor()
            cursor.execute("PRAGMA foreign_keys = ON")
            cursor.execute("SELECT ID_COUNTRY from COUNTRY WHERE COUNTRY=?", (restaurant_dict["country"],))
            country_id = cursor.fetchone()
            if not country_id:
                self.insert("country", {"country": restaurant_dict["country"]})
                country_id = [cursor.lastrowid]
            country_id = country_id[0]

            cursor.execute("SELECT ID_POSTAL_CODE from POSTAL_CODE WHERE POSTAL_CODE=?", (restaurant_dict["postal_code"],))
            postal_code_id = cursor.fetchone()
            if not postal_code_id:
                self.insert("postal_code", {"postal_code": restaurant_dict["postal_code"]})
                postal_code_id = [cursor.lastrowid]
            postal_code_id = postal_code_id[0]


            cursor.execute("SELECT ID_CITY from CITY WHERE CITY_NAME=?", (restaurant_dict["city"],))
            city_id = cursor.fetchone()
            if not city_id:
                self.insert("city", {"city_name": restaurant_dict["city"], "ID_postal_code": postal_code_id})
                city_id = [cursor.lastrowid]
            city_id = city_id[0]

            restaurant_dict["restaurant_name"] = restaurant_dict.pop("name")
            restaurant_dict["city"] = city_id
            restaurant_dict["postal_code"] = postal_code_id
            restaurant_dict["country"] = country_id 

            self.insert("restaurant", restaurant_dict)
            restaurant_id = self.fetch_Foreignkey("restaurant", "restaurant_name", restaurant_dict["restaurant_name"])

            for desc, url in restaurant["photos"][0].items():
                photo_dict = {
                'ID_RESTAURANT': restaurant_id[0],
                'DESCRIPTION': desc,
                'URL': url
                }
                self.insert("photos", photo_dict)

            for review in restaurant["reviews"].iloc[0]:
                r_dict = {}

                r_dict["ID_RESTAURANT"] = restaurant_id[0]
                r_dict["REVIEW_DAY"] = review["review_date"]["day"]
                r_dict["review_month"] = review["review_date"]["month"]
                r_dict["review_year"] = review["review_date"]["year"]
                # Invalid values for review["visit_date"]["year"] indicate the fields were missing
                if re.match(r"\d{4}", review["visit_date"]["year"]):
                    r_dict["visit_month"] = review["visit_date"]["month"]
                    r_dict["visit_year"] = review["visit_date"]["year"]
                    r_dict["visit_context"] = review["visit_context"]
                else:
                    r_dict["visit_month"] = None
                    r_dict["visit_year"] = None
                    r_dict["visit_context"] = None
                db_tripadvisor.insert("REVIEWS", r_dict)


    def fetch_Foreignkey(self, table_name, var_name, value):
        with sqlite3.connect(self.PATH_TO_DB) as conn:
            cursor = conn.cursor()
            cursor.execute("PRAGMA foreign_keys = ON")
            cursor.execute(f"SELECT ID_{table_name} from {table_name} WHERE {var_name}=?", (value,))
            FK = cursor.fetchone()
        return FK

db_tripadvisor = DButils(path="./", filename="tripadvisor.db", exists_ok=True)

In [3]:
def drop_all_tables(db_path: str):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Get the list of all table names from sqlite_master
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    # Drop each table
    for table in tables:
        table_name = table[0]
        print(f"Dropping table: {table_name}")
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    
    # Commit the changes and close the connection
    conn.commit()
    conn.close()


country_schema = {
    "ID_COUNTRY": "INTEGER PRIMARY KEY",
    "COUNTRY": "TEXT"
}

postal_code_schema = {
    "ID_POSTAL_CODE": "INTEGER PRIMARY KEY",
    "POSTAL_CODE": "TEXT"
}

city_schema = {
    "ID_CITY": "INTEGER PRIMARY KEY",
    "ID_COUNTRY": "INTEGER REFERENCES COUNTRY(ID_COUNTRY)",
    "ID_POSTAL_CODE": "INTEGER REFERENCES CODE_POSTAL(ID_POSTAL_CODE)",
    "CITY_NAME": "TEXT"
}

restaurant_schema = {
    "ID_RESTAURANT": "INTEGER PRIMARY KEY",
    "SOURCE_PAGE": "TEXT UNIQUE",
    "RESTAURANT_NAME": "TEXT",
    "CLAIMED": "BOOLEAN",
    "PRICE_RANGE": "TEXT",
    "ADDRESS": "TEXT",
    "POSTAL_CODE": "INTEGER REFERENCES CODE_POSTAL(ID_POSTAL_CODE)",
    "CITY": "INTEGER REFERENCES CITY(ID_CITY)",
    "COUNTRY": "INTEGER REFERENCES COUNTRY(ID_COUNTRY)",
    "PHONE_NUMBER": "TEXT",
    "OPENING_HOURS": "TEXT",
    "TRAVELERS_CHOICE": "TEXT",
    "DETAILED_RATING": "TEXT",
    "DETAILS": "TEXT",
    "REVIEW_NUMBER": "INTEGER",
    "OVERALL_RATING": "REAL",
    "RANKING": "TEXT"
}

photos_schema = {
    "ID_PHOTO": "INTEGER PRIMARY KEY",
    "ID_RESTAURANT": "INTEGER REFERENCES RESTAURANT(ID_RESTAURANT)",
    "DESCRIPTION": "TEXT",
    "URL": "TEXT"
}

reviews_schema = {
    "ID_REVIEW": "INTEGER PRIMARY KEY",
    "ID_RESTAURANT": "INTEGER REFERENCES RESTAURANT(ID_RESTAURANT)",
    "N_CONTRIB": "INTEGER",
    "REVIEW_TITLE": "TEXT",
    "REVIEW_BODY": "TEXT",
    "REVIEW_SCORE": "REAL",
    "REVIEW_DAY": "TEXT",
    "REVIEW_MONTH": "TEXT",
    "REVIEW_YEAR": "TEXT",
    "VISIT_MONTH": "TEXT",
    "VISIT_YEAR": "TEXT",
    "VISIT_CONTEXT": "TEXT"
}

schemas = {
    "country": country_schema,
    "postal_code": postal_code_schema,
    "city": city_schema,
    "restaurant": restaurant_schema,
    "photos": photos_schema,
    "reviews": reviews_schema
}


# Pour effacer les tables en lien avec la table restaurant.
drop_all_tables(db_tripadvisor.PATH_TO_DB)
db_tripadvisor.create_table("country", country_schema)
db_tripadvisor.create_table("city", city_schema)
db_tripadvisor.create_table("postal_code", postal_code_schema)
db_tripadvisor.create_table("restaurant", restaurant_schema)
db_tripadvisor.create_table("photos", photos_schema)
db_tripadvisor.create_table("reviews", reviews_schema)



Dropping table: COUNTRY
Dropping table: CITY
Dropping table: POSTAL_CODE
Dropping table: RESTAURANT
Dropping table: PHOTOS
Dropping table: REVIEWS


True

In [4]:
parquet_files = [file for file in os.listdir() if ".parquet" in file]
for file in parquet_files:
    print(f"adding{file}")
    df = pd.read_parquet(file)
    db_tripadvisor.insert_restaurant(df)

addingAromatic Restaurant.parquet
addingBLO Restaurant.parquet
addingL'Argot.parquet
addingL'Institut Restaurant.parquet
addingLa Gargotte.parquet
addingLe Boeuf D'argent.parquet
addingLe Casse Museau.parquet
addingMama Restaurant Lyon.parquet
addingRestaurant Le Musée.parquet


In [5]:
with sqlite3.connect(db_tripadvisor.PATH_TO_DB) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * from restaurant")
    for item in cursor.fetchall():
        print(item)


(1, 'https://www.tripadvisor.fr/Restaurant_Review-g187265-d12337867-Reviews-Aromatic_Restaurant-Lyon_Rhone_Auvergne_Rhone_Alpes.html', 'Aromatic Restaurant', b'\x01', 'Française', "15 Rue du Chariot d'Or", 0, 0, 0, '+33 4 78 23 73 61', '["Dimanche", "Fermé ", "Lundi", "Fermé ", "Mardi", "12:00-14:0018:30-20:30", "Mercredi", "12:00-14:0018:30-20:30", "Jeudi", "12:00-14:0018:30-20:30", "Vendredi", "12:00-14:0018:30-20:30", "Samedi", "12:00-14:0018:30-20:30"]', '2024', '{"Excellent": 934, "Horrible": 5, "Moyen": 41, "Médiocre": 3, "Très bon": 135}', '{"CUISINES": "[\\"Française, \\", \\"Européenne\\"]", "FONCTIONNALITÉS": "[\\"Accessible en fauteuil roulant\\", \\"Cartes bancaires acceptées\\", \\"Chaises hautes disponibles\\", \\"Places assises\\", \\"Réservations\\", \\"Salle privée\\", \\"Sert de l\'alcool\\", \\"Service de table\\", \\"Terrasse\\"]", "Régimes spéciaux": "[\\"Convient aux végétariens\\"]", "Types de repas": "[\\"Déjeuner, \\", \\"Dîner\\"]"}', b'^\x04\x00\x00\x00\x00\x

In [6]:
with sqlite3.connect(db_tripadvisor.PATH_TO_DB) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * from photos")
    for item in cursor.fetchall():
        print(item)

(1, 1, "Découvrez le menu du mois de Décembre à l'Aromatic !", 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/a1/5d/a7/decouvrez-le-menu-du.jpg?w=900&h=500&s=1')
(2, 1, 'Foie gras du Sud-Ouest poêlé et haricots coco confits à la saucisse de Montbéliard', 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/17/5f/a8/f8/foie-gras-du-sud-ouest.jpg?w=900&h=500&s=1')
(3, 1, 'Intérieur', 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/0f/d3/f2/aromatic.jpg?w=300&h=-1&s=1')
(4, 1, 'Menu', 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/ce/53/f2/menu.jpg?w=300&h=200&s=1')
(5, 1, 'Plats', 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/0e/d7/1b/dorade-de-corse.jpg?w=300&h=200&s=1')
(6, 1, "Une terrasse à la Croix-Rousse pour déjeuner et dîner à l'extérieur. ", 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/06/17/34/une-terrasse-a-la-croix.jpg?w=900&h=-1&s=1')
(7, 2, 'EXCELLENT', 'https://dynamic-media-cdn.tripadvisor.com/media/ph

In [7]:
with sqlite3.connect(db_tripadvisor.PATH_TO_DB) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * from reviews")
    for item in cursor.fetchall():
        print(item)

(1, 1, 1, 'Super tout', 'Super service, super entrée, super plat et super dessert ! Produits frais et extras , nous nous sommes réglées encore une fois !', 5.0, '21', 'décembre', '2024', 'déc.', '2024', 'couples')
(2, 1, 1, 'Excellente découverte', 'Un menu découverte fantastique avec des produits parfaitement travaillés.\n\nUn accord mets vins plein de découverte.\n\nUne ambiance chic mais décontractée \nUne équipe aux petits soins \n\nBref un super resto à faire en couple, entre amis ou même en famille pour un super RQP.\nMerci Aromatic', 5.0, '3', 'décembre', '2024', 'déc.', '2024', 'couples')
(3, 1, 45, 'Une belle découverte', "L'accueil était chaleureux et le service impeccable.\nLa carte est variée, sans excès et les plats de qualité.\nLes vins, même au verre, sont de très belle facture.\nEt tout cela pour un prix somme toute raisonnable.\nA ne pas manquer, hormis peut être le café décevant.....mais il faut bien une petite fausse note !", 4.0, '1', 'décembre', '2024', 'nov.', '20